#  📌 협업 필터링 애니메이션 추천(SVD)

### ✏️ DB 연결

In [1]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.animation
dbcol_detail = db.ani_info
dbcol_review = db.review

### ✏️ import

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

### ✏️ DataFrame 가져오고 columns 수정

In [3]:
# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1, "series_id":1}))
ani_df.drop('_id', axis = 1, inplace = True)
ani_df.columns = ["ani_id", "ani_name", "series_id"]
# ani_df

In [4]:
# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["profile", "animation", "score"]]
rating_df.columns = ["user_id", "ani_id", "score"]
# rating_df = rating_df.sort_values('user_id')
# rating_df

### ✏️ Matrix로 변경

In [5]:
# 사용자 - 애니 pivot table 생성
user_ani_ratings_df = rating_df.pivot(
    index='user_id',
    columns='ani_id',
    values='score'
).fillna(0)
# user_ani_ratings_df

In [6]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = user_ani_ratings_df.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-애니에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

# matrix
# matrix.shape #(91423, 4586)
# user_ratings_mean.shape #(91423,)
# matrix_user_mean.shape #(91423, 4586)
# pd.DataFrame(matrix_user_mean, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns).head()

여기까지 진행하며 사용자가 애니에 대해 남긴 평점을 변경

### ✏️ SVD를 이용해 Matrix Factorization
- spicy를 이용해서 Truncated SVD를 구하기
- scikit learn에서 제공하는 Truncated SVD는 U, Sigma, Vt 반환 값을 제공x
- spicy에서 제공하는 TruncatedSVD는 scipy.sparse.linalg.svds를 이용하면 제공o

In [7]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 12)
# print(U.shape) #(91423, 12)
# print(sigma.shape) #(12,)
# print(Vt.shape) #(12, 4586)

In [8]:
# 위는 0이 아닌 값만 포함되었기에 0이 포함된 대칭행렬로 diag를 이용해 변환
sigma = np.diag(sigma)
# sigma.shape #(12, 12)
# sigma[0]
# sigma[1]

In [9]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 아까 평균을 빼었으니 다시 사용자 평균 rating 더한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns)
# df_svd_preds.head()
# df_svd_preds.shape #(91423, 4586)
# df_svd_preds.loc[15].sort_values(ascending=False)

### ✏️ SVD를 활용한 애니메이션 추천
- 사용자 아이디에 SVD로 나온 결과의 애니 평점이 가장 높은 데이터 순으로 정렬
- 사용자가 본 애니를 제외
- 사용자가 안 본 애니에서 평점이 높은 것을 반환
- 같은 시리즈인 것이 이미 추천되었다면 추가 안 함
- 30개의 애니메이션 아이디를 반환

In [10]:
id_to_idx = dict(zip(ani_df["ani_id"], ani_df.index))
idx_to_series = dict(zip(ani_df.index, ani_df["series_id"]))
id_to_series = dict(zip(ani_df["ani_id"], ani_df["series_id"]))

In [11]:
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendation_list = list(recommendations.sort_values(userId, ascending = False)['ani_id'])

    # 추천 애니메이션 저장 리스트
    recommendation = []
    
    # 시리즈 저장 리스트
    series_list = []
    
    for id in recommendation_list:
        
        # 선택한 애니메이션의 인덱스 가져옴
        idx = id_to_idx[id]

        # 해당 인덱스의 시리즈 아이디 가져옴
        series = idx_to_series[idx]
        
        # 시리즈가 미리 추가되어 있다면 결과에 추가하지 않음
        if series not in series_list:
            recommendation.append(id)
            if np.isnan(series) == False:
                series_list.append(series)
        
        # 30개가 추천되면 종료
        if len(recommendation) == 30:
            return recommendation

In [12]:
input_user_id = 4523846

In [13]:
recommendation_list = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df)
# recommendation_list

In [14]:
recommendation_list

[40085,
 40630,
 40510,
 39070,
 23661,
 39081,
 40794,
 40153,
 39992,
 40163,
 39985,
 38921,
 40159,
 40790,
 40829,
 39856,
 39745,
 40276,
 40394,
 40272,
 39648,
 40682,
 40266,
 39847,
 40533,
 40261,
 40692,
 40693,
 39731,
 40181]

### ✏️ 참고

##### 이미 본 애니들과 추천 애니들을 보기 위한 코드
```python
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df, num_recommendations):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {userId: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]  

    return return user_history, recommendations

already_rated, predictions = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df, 30)
print(already_rated)
print(predictions)
```
#####  num_recommendations개를 추천받는 코드
```python
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df, num_recommendations):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendation_list = list(recommendations.sort_values(userId, ascending = False)['ani_id'])[:num_recommendations]

    return recommendation_list
```
##### 시리즈 체크 코드
```python
id_to_idx = dict(zip(ani_df["ani_id"], ani_df.index))
idx_to_series = dict(zip(ani_df.index, ani_df["series_id"]))
id_to_series = dict(zip(ani_df["ani_id"], ani_df["series_id"]))

def series_check(recommendation_list):
    # 추천 애니메이션 저장 리스트
    recommendation = []
    # 시리즈 저장 리스트
    series_list = []
    
    for id in recommendation_list:
        # 선택한 애니메이션의 인덱스 가져옴
        idx = id_to_idx[id]

        # 해당 인덱스의 시리즈 아이디 가져옴
        series = idx_to_series[idx]
        
        if series not in series_list:
            recommendation.append(id)
            if np.isnan(series) == False:
                series_list.append(series)

    return recommendation
```